In [ ]:
# import relevant libraries
import os
import h5py
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from variable_share import y

from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis

from sklearn.cluster import MiniBatchKMeans

We first picked a sample song to explore the avaliable acoustic features for song clustering. Since all the acoustic information for a song is stored in the 'analysis' branch, we examined only the 'analysis' part 

In [3]:
path = 'D:/songdata/MillionSongSubset/A/O/G/TRAOGTM128F931FABE.h5'
f = h5py.File(path, 'r')

dict(f['analysis'])

{'bars_confidence': <HDF5 dataset "bars_confidence": shape (1000,), type "<f8">,
 'bars_start': <HDF5 dataset "bars_start": shape (1000,), type "<f8">,
 'beats_confidence': <HDF5 dataset "beats_confidence": shape (1000,), type "<f8">,
 'beats_start': <HDF5 dataset "beats_start": shape (1000,), type "<f8">,
 'sections_confidence': <HDF5 dataset "sections_confidence": shape (14,), type "<f8">,
 'sections_start': <HDF5 dataset "sections_start": shape (14,), type "<f8">,
 'segments_confidence': <HDF5 dataset "segments_confidence": shape (1506,), type "<f8">,
 'segments_loudness_max': <HDF5 dataset "segments_loudness_max": shape (1506,), type "<f8">,
 'segments_loudness_max_time': <HDF5 dataset "segments_loudness_max_time": shape (1506,), type "<f8">,
 'segments_loudness_start': <HDF5 dataset "segments_loudness_start": shape (1506,), type "<f8">,
 'segments_pitches': <HDF5 dataset "segments_pitches": shape (1506, 12), type "<f8">,
 'segments_start': <HDF5 dataset "segments_start": shape (15

Feature names ending with '_confidence' represent the detection accuracy of the algorithm, ranging from 0 to 1, where a higher value indicates more reliable information. Here for convenience, we assumed that all acoustic feature-related confidence scores were generally reliable. Most of the acoustic features are stored in a 1-D array format, except for pitch- and timbre-related features, which are in a 2-D array format. While we acknowledge that these are important attributes of a song, we dropped them two to maintain alignment in the training data.

After exploring the sample song, we proceeded to extract data for all 10,000 songs. And as mentioned in the 'draft', we needed to calculate relevant summary statistics for each acoustic feature and then perform clustering based on them. We wrote a reusable function to accomplish this.

In [4]:
directory = "D:/songdata/MillionSongSubset"

def read_from_filepaths(directory, feature_name): #create a list containing each song's specific acoustic feature array 
    file_paths = []
    
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.h5'):
                file_paths.append(dirpath+'/'+filename)

    file_paths = [file_path.replace('\\', '/') for file_path in file_paths] # filepath of all the songs

    acoustic_feature_lis = []

    for file_path in file_paths:
        file = h5py.File(file_path, 'r')
        acoustic_feature_lis.append(np.array(file['analysis'][feature_name]))
    
    return acoustic_feature_lis

In [5]:
def seq_stats(x): # statistics: mean, standard deviation, interquantile range, coefficient of variation, skewness, kurtosis, quantile, length
    x = np.asarray(x)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.array([np.nan]*10)
    q10, q50, q90 = np.percentile(x, [10,50,90])
    iqr = np.percentile(x, 75) - np.percentile(x, 25)
    cv = np.std(x) / (np.mean(x)+1e-8)

    return np.array([np.mean(x), np.std(x), iqr, cv, skew(x), kurtosis(x), q10, q50, q90, len(x)])


def hist_feat(x, bins=20): #instance number in each bin, we set 20 bins
    if len(x)==0: return np.zeros(bins)
    h, _ = np.histogram(x, bins=bins, density=True)
    return h

def cal_stat_matrix(acoustic_array): # create matrix whose columns are all summary statistics
    rows = []

    for b in acoustic_array:  # list[np.ndarray]
        if b is None or len(b)<2:
            rows.append(np.zeros(10+20))  # 占位
            continue
        
        d = np.diff(b)
        d = d[d>0]
        if d.size == 0:
            rows.append(np.zeros(10+20))
            continue
        d = d / (np.median(d)+1e-8)
        rows.append(np.concatenate([seq_stats(d), hist_feat(d, bins=20)])) # 30 summary statistics in total

    X_stat = np.vstack(rows)  # (n_songs, ~30)
    X_stat = np.nan_to_num(X_stat, nan=0.0, posinf=0.0, neginf=0.0)
    return X_stat

After obtaining the statistics matrix, we performed K-Means clustering. The reusable function is shown below, and we used MiniBatchKMeans to speed up computation.

In [6]:
def build_cluster_oof_features(X_stats, y, k=32, n_splits=5): # cluster number
    n = len(y)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_dist  = np.zeros((n, k)) # distance to core of each cluster, each song
    oof_label = np.zeros(n, dtype=int) # cluster label, each song

    for tr, va in kf.split(X_stats): # training, validation sets
        scaler = StandardScaler().fit(X_stats[tr])
        Xz_tr = scaler.transform(X_stats[tr])
        Xz_va = scaler.transform(X_stats[va])

        km = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048, n_init="auto").fit(Xz_tr)

        oof_dist[va]  = km.transform(Xz_va)  
        oof_label[va] = km.predict(Xz_va)

    return (oof_dist, oof_label)

# Song acoutic feature

We then performed song clustering on a feature-by-feature basis.

## bars_start

In [7]:
bar_array = read_from_filepaths(directory, "bars_start")
X_stat = cal_stat_matrix(bar_array)

In [8]:
array_cluster_bars = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 系统找不到指定的文件。
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\miniconda3\envs\pydata-book\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\pydata-book\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\pydata-book\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "d:\miniconda3\envs\pydata-book\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

In [9]:
array_cluster_bars[0].shape, array_cluster_bars[1].shape #the distance of a song to each cluster core + clustering label predicted from validation sets

((10000, 32), (10000,))

## beats_start

In [10]:
beat_array = read_from_filepaths(directory, "beats_start")
X_stat = cal_stat_matrix(beat_array)

In [11]:
array_cluster_beats = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## sections_start

In [12]:
section_array = read_from_filepaths(directory, "sections_start")
X_stat = cal_stat_matrix(section_array)

In [13]:
array_cluster_sections = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## segments_loudness_max

In [14]:
loudness_array = read_from_filepaths(directory, "segments_loudness_max")
X_stat = cal_stat_matrix(loudness_array)

array_cluster_loudness = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## segments_loudness_max_time

In [15]:
loudness_time_array = read_from_filepaths(directory, "segments_loudness_max_time")
X_stat = cal_stat_matrix(loudness_time_array)

array_cluster_loudness_time = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## segments_loudness_start

In [16]:
loudness_start_array = read_from_filepaths(directory, "segments_loudness_start")
X_stat = cal_stat_matrix(loudness_start_array)

array_cluster_loudness_start = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## segments_start

In [17]:
segments_start_array = read_from_filepaths(directory, "segments_start")
X_stat = cal_stat_matrix(segments_start_array)

array_cluster_segments_start = build_cluster_oof_features(X_stat, y)

d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warnings.warn(
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  warn

## tatums_start

In [18]:
tatums_start_array = read_from_filepaths(directory, "tatums_start")
X_stat = cal_stat_matrix(tatums_start_array)

array_cluster_tatums_start = build_cluster_oof_features(X_stat, y)

C:\Users\fuyuz\AppData\Local\Temp\ipykernel_13804\3304920897.py:10: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return np.array([np.mean(x), np.std(x), iqr, cv, skew(x), kurtosis(x), q10, q50, q90, len(x)])
d:\miniconda3\envs\pydata-book\Lib\site-packages\numpy\lib\_histograms_impl.py:902: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
d:\miniconda3\envs\pydata-book\Lib\site-packages\numpy\lib\_histograms_impl.py:902: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
d:\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\cluster\_kmeans.py:1952: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5632 or by setting the environment variable OMP_NUM_THREADS=8
  war

In total, we used 8 acoustic features, each associated with a clustering label array, and we saved all of these arrays.

In [19]:
cluster_label_bars = array_cluster_bars[1].reshape(-1, 1)
cluster_label_beats = array_cluster_beats[1].reshape(-1,1)
cluster_label_loudness = array_cluster_loudness[1].reshape(-1, 1)
cluster_label_loudness_time = array_cluster_loudness_time[1].reshape(-1, 1)

cluster_label_sections = array_cluster_sections[1].reshape(-1, 1)
cluster_label_loudness_start = array_cluster_loudness_start[1].reshape(-1,1)
cluster_label_segments_start = array_cluster_segments_start[1].reshape(-1, 1)
cluster_label_tatums_start = array_cluster_tatums_start[1].reshape(-1, 1)

In [20]:
np.save("cluster_label_bars", cluster_label_bars)
np.save("cluster_label_beats", cluster_label_beats)
np.save("cluster_label_loudness", cluster_label_loudness)
np.save("cluster_label_loudness_time", cluster_label_loudness_time)

np.save("cluster_label_sections", cluster_label_sections)
np.save("cluster_label_loudness_start", cluster_label_loudness_start)
np.save("cluster_label_segments_start", cluster_label_segments_start)
np.save("cluster_label_tatums_start", cluster_label_tatums_start)